In [1]:
# MNISTデータセットをインポート
from keras.datasets import mnist
# MNISTデータセットの読み込み
(x_trains, y_trains), (x_tests, y_tests) = mnist.load_data()

Using TensorFlow backend.


In [2]:
print(x_trains.shape)  # (60000, 28, 28) 訓練データ用の画像が60000
print(y_trains.shape)  # 出力：(60000,) 訓練データの正解ラベルが60000
print(x_tests.shape)   # 出力：(10000, 28, 28) テスト用の画像が10000
print(y_tests.shape)   # 出力：(10000,) テスト用の正解ラベルが10000

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
print(y_trains)  # 出力：[5 0 4 ..., 5 6 8]

[5 0 4 ... 5 6 8]


In [4]:
# keras.utilsからnp_utilsをインポート
from keras.utils import np_utils

# 正解ラベルのクラスの数
classes = 10

# 訓練データ
# 60000x28x28の3次元配列を60000×784の2次元配列に変換
x_trains = x_trains.reshape(60000, 784)
# データを255で割って0から1.0の範囲に変換
x_trains = x_trains / 255
# 正解ラベルの数
classes = 10

# 正解ラベルをワンホット表現に変換
y_trains = np_utils.to_categorical(y_trains, classes)

# テストデータ
# 10000x28x28の3次元配列を10000×784の2次元配列に変換
x_tests = x_tests.reshape(10000, 784)
# データを255で割って0から1.0の範囲に変換
x_tests = x_tests / 255
# 正解ラベルをワンホット表現変換
y_tests = np_utils.to_categorical(y_tests, classes)

In [5]:
# keras.modelsからSequentialをインポート
from keras.models import Sequential
# keras.layersからDense、Activationをインポート
from keras.layers import Dense, Activation
# keras.optimizersからAdamをインポート
from keras.optimizers import Adam

model = Sequential()                 # Sequentialのインスタンス化
model.add(Dense(200,                 # 隠れ層のニューロン数は200
                input_dim=784,       # 入力層のニューロン数は784
                activation='sigmoid' # 活性化関数はシグモイド
               ))
model.add(Dense(10,                  # 出力層のニューロン数は10
                activation='softmax' # 活性化関数はソフトマックス
               ))
model.compile(
    loss='categorical_crossentropy', # 損失の基準は交差エントロピー誤差
    optimizer=Adam(),                # 学習方法をAdamにする
    metrics=['accuracy']             # 学習評価として正解率を指定
    )
model.summary() # ニューラルネットワークのサマリー（概要）を出力

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
Total params: 159,010
Trainable params: 159,010
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 学習を行って結果を出力
history = model.fit(x_trains,            # 訓練データ
                    y_trains,            # 正解ラベル
                    epochs=5,            # 学習を繰り返す回数
                    batch_size=100,      # 勾配計算に用いるミニバッチの数
                    verbose=1,           # 学習の進捗状況を出力する
                    validation_data=(
                        x_tests, y_tests # テストデータの指定
                    ))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 107us/step - loss: 0.4919 - acc: 0.8771 - val_loss: 0.2751 - val_acc: 0.9197
Epoch 2/5
60000/60000 [==============================] - 5s 85us/step - loss: 0.2456 - acc: 0.9294 - val_loss: 0.2080 - val_acc: 0.9411
Epoch 3/5
60000/60000 [==============================] - 5s 82us/step - loss: 0.1917 - acc: 0.9452 - val_loss: 0.1755 - val_acc: 0.9481
Epoch 4/5
60000/60000 [==============================] - 5s 84us/step - loss: 0.1559 - acc: 0.9554 - val_loss: 0.1471 - val_acc: 0.9569
Epoch 5/5
60000/60000 [==============================] - 6s 103us/step - loss: 0.1315 - acc: 0.9626 - val_loss: 0.1274 - val_acc: 0.9617


In [7]:
# 学習したモデルと重みと履歴の保存
# モデルの保存
open('model.json',"w").write(model.to_json())

# 学習済みの重みを保存
model.save_weights('weight.h5')

In [12]:
from keras.models import model_from_json
# モデルの読み込み
model_r = model_from_json(open('model.json', 'r').read())

# 重みの読み込み
model_r.load_weights('weight.h5')

In [9]:
model_r.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               157000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2010      
Total params: 159,010
Trainable params: 159,010
Non-trainable params: 0
_________________________________________________________________


In [10]:
model_r.compile(
    loss='categorical_crossentropy', # 損失の基準は交差エントロピー誤差
    optimizer=Adam(),                # 学習方法をAdamにする
    metrics=['accuracy']             # 学習評価として正解率を指定
    )
score = model_r.evaluate(x_tests, y_tests, batch_size=128, verbose=1)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

10000/10000 [==============================] - 1s 70us/step
Test loss : 0.12738918604552746
Test accuracy : 0.9617
